# Chapter 2. 动态规划
动态规划是满足最优性原理的问题的解决思路。它衍生出了一系列问题-算法的组合。

其关键在于，通过阶段的分割，做出充分的剪枝，从而把指数复杂度的暴力搜索化为多项式复杂度的迭代，得出一个最优结果。

最大的难点一般是状态的设计。一般而言，需要设计出“在XX坐标（1-2D，与状态节点和迭代轮次相关，对应某限制条件）
下的最优值”这样的状态，逐步松弛限制到全集；决策变量一般是显然的。

有人认为将线性规划之类归为“静态规划”，与动态规划相对立。私以为不然。规划问题是一类问题，动态规划是一类算法思路，
动态规划是用来设计解决规划问题的算法的。例如背包问题，本质上是整数线性规划，可以用线性规划算法解决，时间复杂度也在可接受的
$O(n^{2.38})$ ；但通过背包 DP，可以优化算法复杂度到 $O(n^2)$ 。

从实用主义角度而言，在数学建模的场景中，建议熟悉常见的 DP 模型，看到的时候能反应出来并搓出来代码就好。除非是 OIer，
构造新 DP 算法（证明，搓代码，debug）的时间，都够暴力搜索算法搓完算完了。算不完？大概率是模型太复杂了，优化模型去！

## 分配问题
狭义上这类问题的特征是：指标为总成本/利润，决策变量是用来分配的；则我们以 后 k 周期的总成本/利润为阶段指标，剩余量为伪常量，
利用（通常带有 minmax 的）状态转移函数，倒推阶段指标表达式。

广义上，任何以总和/积为唯一约束的场景都可以这么算，只要把剩余量作为伪常量、存在转移方程即可。

由于**分配**的存在，当后面的决策值确定了，总周期就被拆解成了最后一个和前面的，分配总量也被拆解出了子总量，从而完成了后续决策的计算上“解耦”。
这是纯粹的数学技巧。

例：
$$
\begin{align*}
\max\qquad & z=u_1u_2^2u_3 \\ 
\text{s.t.}\qquad & \left\{
    \begin{array}{l}
    u_1+u_2+u_3=c\qquad(c\gt0)\\
    u_i\ge0\qquad i=1,2,3\\
    \end{array}
\right.\\
\end{align*}
$$

由于存在总和约束，可以用分配算法算。令 $x_k=\sum_{i=k}^nu_i$ ，利用 $x_{k+1}=x_k-u_k$

In [54]:
import sympy
from scipy.optimize import minimize_scalar
u=sympy.symbols("u1:4")
xi=sympy.symbols("x_i")

z=u[0]*u[1]*u[1]*u[2] # u1 u2^2 u3

for i in range(2,-1,-1):
    z=z.subs(xi, xi-u[i])
    # normalize
    zt=z.subs(xi,1)
    for j in range(i):
        zt=zt.subs(u[j],1)
    # minimize
    coeff=minimize_scalar(
        lambda x:-sympy.lambdify(u[i], zt)(x),
        bounds=[0,1]
    ).x
    z=z.subs(u[i], coeff*xi)
# result
z*64

0.99999403913243*x_i**4

## 背包问题
看 [背包 DP - OI Wiki](https://oi.wiki/dp/knapsack/) 吧